# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6968398400>
├── 'a' --> tensor([[-0.1583,  0.6809, -1.6269],
│                   [-1.4911,  0.9222,  0.4929]])
└── 'x' --> <FastTreeValue 0x7f6968398c40>
    └── 'c' --> tensor([[-0.1458, -1.5803, -0.8079,  2.0455],
                        [-1.1734, -1.0550, -0.7487,  1.3342],
                        [ 0.4668,  0.1016,  0.9536,  0.0891]])

In [4]:
t.a

tensor([[-0.1583,  0.6809, -1.6269],
        [-1.4911,  0.9222,  0.4929]])

In [5]:
%timeit t.a

66.2 ns ± 0.0424 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6968398400>
├── 'a' --> tensor([[-0.8708,  0.5917, -1.0340],
│                   [ 0.8504, -0.8276,  1.2894]])
└── 'x' --> <FastTreeValue 0x7f6968398c40>
    └── 'c' --> tensor([[-0.1458, -1.5803, -0.8079,  2.0455],
                        [-1.1734, -1.0550, -0.7487,  1.3342],
                        [ 0.4668,  0.1016,  0.9536,  0.0891]])

In [7]:
%timeit t.a = new_value

75 ns ± 0.0178 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1583,  0.6809, -1.6269],
               [-1.4911,  0.9222,  0.4929]]),
    x: Batch(
           c: tensor([[-0.1458, -1.5803, -0.8079,  2.0455],
                      [-1.1734, -1.0550, -0.7487,  1.3342],
                      [ 0.4668,  0.1016,  0.9536,  0.0891]]),
       ),
)

In [10]:
b.a

tensor([[-0.1583,  0.6809, -1.6269],
        [-1.4911,  0.9222,  0.4929]])

In [11]:
%timeit b.a

51.2 ns ± 0.0368 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1720,  1.1860,  0.6991],
               [ 0.2452, -0.1526, -0.8988]]),
    x: Batch(
           c: tensor([[-0.1458, -1.5803, -0.8079,  2.0455],
                      [-1.1734, -1.0550, -0.7487,  1.3342],
                      [ 0.4668,  0.1016,  0.9536,  0.0891]]),
       ),
)

In [13]:
%timeit b.a = new_value

482 ns ± 0.13 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

866 ns ± 0.86 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 45.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 207 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 1.28 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f69683f8df0>
├── 'a' --> tensor([[[-0.1583,  0.6809, -1.6269],
│                    [-1.4911,  0.9222,  0.4929]],
│           
│                   [[-0.1583,  0.6809, -1.6269],
│                    [-1.4911,  0.9222,  0.4929]],
│           
│                   [[-0.1583,  0.6809, -1.6269],
│                    [-1.4911,  0.9222,  0.4929]],
│           
│                   [[-0.1583,  0.6809, -1.6269],
│                    [-1.4911,  0.9222,  0.4929]],
│           
│                   [[-0.1583,  0.6809, -1.6269],
│                    [-1.4911,  0.9222,  0.4929]],
│           
│                   [[-0.1583,  0.6809, -1.6269],
│                    [-1.4911,  0.9222,  0.4929]],
│           
│                   [[-0.1583,  0.6809, -1.6269],
│                    [-1.4911,  0.9222,  0.4929]],
│           
│                   [[-0.1583,  0.6809, -1.6269],
│                    [-1.4911,  0.9222,  0.4929]]])
└── 'x' --> <FastTreeValue 0x7f68be040520>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 65.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f6968398c70>
├── 'a' --> tensor([[-0.1583,  0.6809, -1.6269],
│                   [-1.4911,  0.9222,  0.4929],
│                   [-0.1583,  0.6809, -1.6269],
│                   [-1.4911,  0.9222,  0.4929],
│                   [-0.1583,  0.6809, -1.6269],
│                   [-1.4911,  0.9222,  0.4929],
│                   [-0.1583,  0.6809, -1.6269],
│                   [-1.4911,  0.9222,  0.4929],
│                   [-0.1583,  0.6809, -1.6269],
│                   [-1.4911,  0.9222,  0.4929],
│                   [-0.1583,  0.6809, -1.6269],
│                   [-1.4911,  0.9222,  0.4929],
│                   [-0.1583,  0.6809, -1.6269],
│                   [-1.4911,  0.9222,  0.4929],
│                   [-0.1583,  0.6809, -1.6269],
│                   [-1.4911,  0.9222,  0.4929]])
└── 'x' --> <FastTreeValue 0x7f69683989a0>
    └── 'c' --> tensor([[-0.1458, -1.5803, -0.8079,  2.0455],
                        [-1.1734, -1.0550, -0.7487,  1.3342],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 49.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.1 µs ± 159 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1583,  0.6809, -1.6269],
                [-1.4911,  0.9222,  0.4929]],
       
               [[-0.1583,  0.6809, -1.6269],
                [-1.4911,  0.9222,  0.4929]],
       
               [[-0.1583,  0.6809, -1.6269],
                [-1.4911,  0.9222,  0.4929]],
       
               [[-0.1583,  0.6809, -1.6269],
                [-1.4911,  0.9222,  0.4929]],
       
               [[-0.1583,  0.6809, -1.6269],
                [-1.4911,  0.9222,  0.4929]],
       
               [[-0.1583,  0.6809, -1.6269],
                [-1.4911,  0.9222,  0.4929]],
       
               [[-0.1583,  0.6809, -1.6269],
                [-1.4911,  0.9222,  0.4929]],
       
               [[-0.1583,  0.6809, -1.6269],
                [-1.4911,  0.9222,  0.4929]]]),
    x: Batch(
           c: tensor([[[-0.1458, -1.5803, -0.8079,  2.0455],
                       [-1.1734, -1.0550, -0.7487,  1.3342],
                       [ 0.4668,  0.1016,  0.9536,  0.0891]],
         

In [26]:
%timeit Batch.stack(batches)

78.4 µs ± 232 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1583,  0.6809, -1.6269],
               [-1.4911,  0.9222,  0.4929],
               [-0.1583,  0.6809, -1.6269],
               [-1.4911,  0.9222,  0.4929],
               [-0.1583,  0.6809, -1.6269],
               [-1.4911,  0.9222,  0.4929],
               [-0.1583,  0.6809, -1.6269],
               [-1.4911,  0.9222,  0.4929],
               [-0.1583,  0.6809, -1.6269],
               [-1.4911,  0.9222,  0.4929],
               [-0.1583,  0.6809, -1.6269],
               [-1.4911,  0.9222,  0.4929],
               [-0.1583,  0.6809, -1.6269],
               [-1.4911,  0.9222,  0.4929],
               [-0.1583,  0.6809, -1.6269],
               [-1.4911,  0.9222,  0.4929]]),
    x: Batch(
           c: tensor([[-0.1458, -1.5803, -0.8079,  2.0455],
                      [-1.1734, -1.0550, -0.7487,  1.3342],
                      [ 0.4668,  0.1016,  0.9536,  0.0891],
                      [-0.1458, -1.5803, -0.8079,  2.0455],
                      [-1.1734, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 166 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

311 µs ± 3.14 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
